# Retrieval augmented generation for political text

This is a basic exploration and blueprint of using retrieval-augmented generation (RAG) for summarizing the perspectives of political parties.

For now, the below code builds a simple RAG chain based on the German Green party's manifesto for the parliament elections in 2021.

## 1. Load text

In [1]:
from langchain_community.document_loaders import TextLoader

raw_document = TextLoader("./data/manifestos/txt/2021-gruene.txt").load()

## 2. Split text to equal chunk size

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(raw_document)
print(splits[0], "\n\n")
print(splits[1], "\n\n")

page_content='Eine Einladung Liebe Wähler*innen, durch Wahlen entscheidet eine Gesellschaft, wer sie sein will. Das  gilt erst recht für diese Bundestagswahl am 26. September. Mit ihr  endet eine Ära und eine neue kann beginnen. Zukunft ist aber nichts,  was uns einfach widerfährt. Sie, liebe Wähler*innen, können mit Ihrer  Stimme selbst entscheiden, welche Richtung sie nimmt. Wir, BÜNDNIS 90 / DIE GRÜNEN, legen mit diesem Programm unser  inhaltliches Angebot an Sie vor. Wir tun dies in einer Zeit des globalen  Ausnahmezustands. Die Pandemie hat uns alle bis ins Mark getroffen.  Sie hat im Guten gezeigt, zu welcher Gemeinsamkeit, Innovationskraft und Widerstandsfähigkeit wir Menschen erreichen können. Sie  hat aber auch die Schwachstellen unserer Gesellschaft schonungslos  offengelegt, und das in einer ohnehin verwundbaren Welt. Die globalen Krisen dieser Zeit – zuallererst die Klimakrise als wahre Menschheitskrise – wirken in unser aller Leben hinein und gefährden Freiheit,  Sicherhei

## 3. Embed text and store in vector database

In [3]:
# from langchain_community.embeddings import OllamaEmbeddings
# embeddings = OllamaEmbeddings()

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
from langchain_community.vectorstores import Chroma

# db = Chroma.from_documents(documents = splits,
#                            embedding = embeddings,
#                            persist_directory = "./chroma_db")
db = Chroma(persist_directory="./chroma_db",
            embedding_function=embeddings)

In [6]:
# Test query
query = "Welche Lösungen schlagen die Grünen zum Klimawandel vor?"
docs = db.similarity_search(query)
print(docs[0].page_content, "\n\n")
print(docs[1].page_content, "\n\n")
print(docs[2].page_content, "\n\n")

treffen oft die  am härtesten, die in schwierigsten Umständen leben. Während wir  um jedes Zehntelgrad weniger an Erderhitzung kämpfen, müssen wir  uns zugleich an diese Veränderungen anpassen. In ländlichen Räumen gilt es insbesondere Land- und Forstwirtschaft, Tourismus und  Fischerei bei der Anpassung zu unterstützen, um Schäden durch Dürren, Ernteausfälle und Waldsterben zu verringern. Unsere Städte wollen wir besser gegen Hitzewellen und Starkregen wappnen – mit Hitzeaktionsplänen und einem Stadtumbau im Großen wie im Kleinen:  mehr Stadtgrün, Bodenentsiegelung, Frischluftschneisen, Gebäudebegrünung, Wasserflächen und öffentliche Trinkbrunnen. Als Schwammstädte sollen sie künftig mehr Wasser aufnehmen, speichern und im  Sommer kühlend wirken. Das erhöht auch die Lebensqualität gerade  für all jene, die sich keinen eigenen Balkon oder Garten leisten können: Dachgärten sind natürliche Klimaanlagen für Wohnungen und  Büros, Parks und Stadtwälder spenden Schatten und frische Luft. 




## 4. Retrieve and generate

### 4.1 Set up LLM chain

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
# Test without retrieval

# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You help citizens understand the agenda of political parties."),
#     ("user", "{input}")
# ])

# chain = prompt | llm | output_parser

# response = chain.invoke({"input": "Welche Lösungen schlagen die Grünen zum Klimawandel vor?"})

# import textwrap
# print(textwrap.fill(response, 80))

### 4.2 Set up retrieval mechanism

In [ ]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 7})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
rag_prompt = ChatPromptTemplate.from_template("""Beantworte die folgende Frage nur auf dem zur Verfügung gestellten Kontext.
                                          
                                          KONTEXT:
                                          {context}

                                          FRAGE: {question}""")

In [ ]:
# from langchain_core.runnables import RunnablePassthrough

# retrieval_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | rag_prompt
#     | llm
#     | output_parser
# )

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context = (lambda x: format_docs(x["context"])))
    | rag_prompt
    | llm
    | output_parser
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

### 4.3 Invoke the chain (generate)

In [ ]:
response = rag_chain_with_source.invoke("Welche Lösungen schlagen die Grünen zum Klimawandel vor?")

In [ ]:
import textwrap
print(textwrap.fill(response["answer"], 80))

In [ ]:
for context_snippet in response["context"]:
    print(textwrap.fill(context_snippet.page_content), 80)
    print("\n\n -------------------- \n\n")